# NFL Big Data Bowl

## Plotting the field

Field plotting method was created by Rob Mulla in [this notebook](https://www.kaggle.com/robikscube/nfl-big-data-bowl-plotting-player-position). I re-use it for plotting player position and heatmap.

In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.patches as patches
pd.set_option('max_columns', 100)

### Function to Create The Football Field

In [ ]:
def create_football_field(linenumbers=True,
                          endzones=True,
                          highlight_line=False,
                          highlight_line_number=50,
                          highlighted_name='Line of Scrimmage',
                          fifty_is_los=False,
                          figsize=(12, 6.33),
                          facecolor='darkgreen'):
    """
    Function that plots the football field for viewing plays.
    Allows for showing or hiding endzones.
    """
    rect = patches.Rectangle((0, 0), 120, 53.3, linewidth=0.1,
                             edgecolor='r', facecolor=facecolor, zorder=0)

    fig, ax = plt.subplots(1, figsize=figsize)
    ax.add_patch(rect)

    plt.plot([10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
              80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
             [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
              53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
             color='black')
    if fifty_is_los:
        plt.plot([60, 60], [0, 53.3], color='gold')
        plt.text(62, 50, '<- Player Yardline at Snap', color='gold')
    # Endzones
    if endzones:
        ez1 = patches.Rectangle((0, 0), 10, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ez2 = patches.Rectangle((110, 0), 120, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ax.add_patch(ez1)
        ax.add_patch(ez2)
    plt.xlim(0, 120)
    plt.ylim(-5, 58.3)
    plt.axis('off')
    if linenumbers:
        for x in range(20, 110, 10):
            numb = x
            if x > 50:
                numb = 120 - x
            plt.text(x, 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='black')
            plt.text(x - 0.95, 53.3 - 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='black', rotation=180)
    if endzones:
        hash_range = range(11, 110)
    else:
        hash_range = range(1, 120)

    for x in hash_range:
        ax.plot([x, x], [0.4, 0.7], color='black')
        ax.plot([x, x], [53.0, 52.5], color='black')
        ax.plot([x, x], [22.91, 23.57], color='black')
        ax.plot([x, x], [29.73, 30.39], color='black')

    if highlight_line:
        hl = highlight_line_number + 10
        plt.plot([hl, hl], [0, 53.3], color='yellow')
        plt.text(hl + 2, 50, '<- {}'.format(highlighted_name),
                 color='yellow')
    return fig, ax

In [ ]:
create_football_field()
plt.show()

## Load dataset

We load dataset from a week (from 1 to 17) to plot data. For example, I loaded week 10 dataset.
- The `x` and `y` columns are player coordinate
- `o`: Player orientation (deg), 0 - 360 degrees
- `dir`: Angle of player motion (deg), 0 - 360 degrees (numeric)
- `nflId`: Player identification number, unique across players
- PlayId is not unique across games, only within the game. So for unique ids you need to use a combination of gameId and playId

In [ ]:
data = pd.read_csv('../input/nfl-big-data-bowl-2021/week10.csv')

### Define `gameId` (game identifier) and `nflId` (player identifier)

In [ ]:
player_id = 496733.0
game_id = 2018110800

### Plotting player position in a `gameId`

In [ ]:
player_data = data.query('gameId == @game_id and nflId == @player_id')

fig, ax = create_football_field()
player_data.plot(x='x', y='y', kind='scatter', ax=ax, color='blue', s=5, legend='Away')
plt.title('Game #{}: Player {} positions'.format(game_id, player_data.loc[0, 'displayName']))
plt.legend()
plt.show()

### Plotting heatmap of player position in a gameId

In [ ]:
player_data = data.query('gameId == @game_id and nflId == @player_id')
player_x_coord = player_data['x']
player_y_coord = player_data['y']

fig, ax = create_football_field(facecolor='none')
sns.kdeplot(player_x_coord, player_y_coord, shade = True,
            shade_lowest=False, color = 'green', n_levels=30, ax=ax)

plt.title('Game #{}: Player {} heatmap'.format(game_id, player_data.loc[0, 'displayName']))
plt.legend()
plt.show()